# Import the mlflow db from the experiments

In [187]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///mlflow.db")
print(db.dialect)
print(db.get_usable_table_names())
#db.run("SELECT * FROM metrics LIMIT 10;")

sqlite
['alembic_version', 'datasets', 'experiment_tags', 'experiments', 'input_tags', 'inputs', 'latest_metrics', 'metrics', 'model_version_tags', 'model_versions', 'params', 'registered_model_aliases', 'registered_model_tags', 'registered_models', 'runs', 'tags', 'trace_info', 'trace_request_metadata', 'trace_tags']


# Using Chain

In [171]:
from langchain.chains import create_sql_query_chain
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.1:latest",temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many model versions are there"})
response

In [172]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:



In [173]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
#chain.invoke({"question": "How many employees are there"})

In [174]:
execute_query.invoke("SELECT COUNT(*) FROM model_versions;")

'[(2,)]'

# Agents

LangChain has a SQL Agent which provides a more flexible way of interacting with SQL Databases than a chain. The main advantages of using the SQL Agent are:

It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table).
It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
It can query the database as many times as needed to answer the user question.
It will save tokens by only retrieving the schema from relevant tables.

In [175]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000026FEFFE3590>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000026FEFFE3590>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000026FEFFE3590>),
 QuerySQLCheckerTool(description='Use this tool to 

In [176]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

# System Prompt

In [177]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0)

In [178]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [180]:
question = "How many model versions are there?"

In [184]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {messages}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
     RunnablePassthrough().assign(query= lambda inputs: agent_executor.invoke(inputs)["messages"][-1].content).assign(result=itemgetter("query")
        | execute_query           
    )
    
    | answer_prompt
    | llm
    | StrOutputParser()    

)

chain.invoke({"messages":"How many model versions are there?"})

'There are 2 model versions.'

Exploring the agents

In [30]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-09-29T18:49:34.8904034Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'sql_db_schema', 'arguments': {'table_names': 'model_versions'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 409150426800, 'load_duration': 41611817900, 'prompt_eval_count': 720, 'prompt_eval_duration': 318930545000, 'eval_count': 20, 'eval_duration': 47578886000}, id='run-7b1b56bf-3f2b-4a5e-ac64-1b3aae8acf0e-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'model_versions'}, 'id': 'f6160ee3-bc3e-45ac-a946-a82fd5915544', 'type': 'tool_call'}], usage_metadata={'input_tokens': 720, 'output_tokens': 20, 'total_tokens': 740})]}}
----
{'tools': {'messages': [ToolMessage(content='\nCREATE TABLE model_versions (\n\tname VARCHAR(256) NOT NULL, \n\tversion INTEGER NOT NULL, \n\tcreation_time BIGINT, \n\tlast_updated_time 